In [1]:
#@title Install GFPGAN (Make Sure you are connected with GPU)
#if you are using local device make sure git installed in your device
#comment if you are on local device #start
!rm -rf /content/GFPGAN
%cd /content/
# end
import os
base_path=os.getcwd()+"/"
!git clone https://github.com/TencentARC/GFPGAN.git
os.chdir(f"{base_path}GFPGAN")
!pip install -r requirements.txt
!python setup.py develop
!pip install realesrgan
!pip install gradio
from IPython.display import clear_output
clear_output()

In [3]:
#@title <-- Tap this if you play on Mobile { display-mode: "form" }

# ### if you are using local device set it with you current path

# ```
# base_path="/content"
# ```
# ctrl+f and find ```base_path```

# like
# ```
# base_path="C:/neuralfalcon/upcale"
# ```
%%html
<b>Press play on the music player to keep the tab alive, then run the cell below</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [7]:
#@title Download Models from Public Links


import requests
import sys
import shutil
def fix():
  url = "https://raw.githubusercontent.com/NeuralFalconYT/GFPGAN-video-upscale/main/degradations.py"
  filename = "/content/degradations.py"
  full_version = sys.version.split(' ')[0]
  major_minor_version = '.'.join(full_version.split('.')[:2])
  basicsr_path=f"/usr/local/lib/python{major_minor_version}/dist-packages/basicsr/data/degradations.py"
  # Send a GET request to the URL
  response = requests.get(url)
  # Check if the request was successful (status code 200)
  if response.status_code == 200:
      # Write the content to a file
      with open(filename, 'wb') as file:
          file.write(response.content)
      try:
        shutil.copy("/content/degradations.py",basicsr_path)
        print(f"Copied to {basicsr_path}")
      except:
        print("Update the 'basicsr_path' variable -- replace '{major_minor_version}' with the current Python version in Google Colab, such as 3.10.")

  else:
      print("Failed to download the file.")
fix()


base_path="/content"
import os
import shutil
from zipfile import ZipFile
from tqdm import tqdm
import urllib
def conditional_download(url, download_file_path):
    print(f"Downloading {os.path.basename(download_file_path)}")
    base_path = os.path.dirname(download_file_path)

    if not os.path.exists(base_path):
        os.makedirs(base_path)

    if os.path.exists(download_file_path):
        os.remove(download_file_path)

    try:
        request = urllib.request.urlopen(url)  # type: ignore[attr-defined]
        total = int(request.headers.get('Content-Length', 0))
    except urllib.error.URLError as e:
        print(f"Error: Unable to open the URL - {url}")
        print(f"Reason: {e.reason}")
        return

    with tqdm(total=total, desc='Downloading', unit='B', unit_scale=True, unit_divisor=1024) as progress:
        try:
            urllib.request.urlretrieve(url, download_file_path, reporthook=lambda count, block_size, total_size: progress.update(block_size))  # type: ignore[attr-defined]
        except urllib.error.URLError as e:
            print(f"Error: Failed to download the file from the URL - {url}")
            print(f"Reason: {e.reason}")
            return

    print(f"Download successful!")
    print(f"URL: {url}")
    print(f"Save at: {download_file_path}")
def extract_zip(zip_file_path,extract_path):
    # Create the extraction directory if it doesn't exist
    os.makedirs(extract_path, exist_ok=True)
    # Open the ZIP file
    with ZipFile(zip_file_path, 'r') as zip_ref:
        # Extract all contents to the specified directory
        zip_ref.extractall(extract_path)
    print(f"Contents of '{zip_file_path}' extracted to '{extract_path}'.")


inference_gfpgan='https://raw.githubusercontent.com/NeuralFalconYT/GFPGAN-video-upscale/refs/heads/main/inference_gfpgan.py'# @param {type: "string"}
conditional_download(inference_gfpgan,f"{base_path}/GFPGAN/inference_gfpgan.py")
# download_image='https://raw.githubusercontent.com/neuralfalcon/GFPGAN-video-upscale/main/Download.png'# @param {type: "string"}
# conditional_download(download_image,f"{base_path}/GFPGAN/Download.png")
gfpgan_model = 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth '  # @param {type: "string"}
if not os.path.exists(f"{base_path}/GFPGAN/experiments/pretrained_models/GFPGANv1.3.pth"):
  conditional_download(gfpgan_model,f"{base_path}/GFPGAN/experiments/pretrained_models/GFPGANv1.3.pth")
RealESRGAN = 'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth'  # @param {type: "string"}
if not os.path.exists(f"{base_path}/GFPGAN/gfpgan/weights/RealESRGAN_x2plus.pth"):
  conditional_download(RealESRGAN,f"{base_path}/GFPGAN/gfpgan/weights/RealESRGAN_x2plus.pth")
detection_Resnet50 = 'https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth'  # @param {type: "string"}
if not os.path.exists(f"{base_path}/GFPGAN/gfpgan/weights/detection_Resnet50_Final.pth"):
  conditional_download(detection_Resnet50,f"{base_path}/GFPGAN/gfpgan/weights/detection_Resnet50_Final.pth")
parsing_parsenet = 'https://github.com/xinntao/facexlib/releases/download/v0.2.2/parsing_parsenet.pth'  # @param {type: "string"}
if not os.path.exists(f"{base_path}/GFPGAN/gfpgan/weights/parsing_parsenet.pth"):
  conditional_download(parsing_parsenet,f"{base_path}/GFPGAN/gfpgan/weights/parsing_parsenet.pth")
from IPython.display import clear_output
clear_output()

In [12]:
#@title Utils
base_path="/content"
import cv2
from tqdm import tqdm
from os import path
import shutil
import os
from IPython.display import clear_output
os.chdir(f"{base_path}/GFPGAN")
def create_directory(directory_path):
    if os.path.exists(directory_path):
        shutil.rmtree(directory_path)
    os.makedirs(directory_path)

def extract_frames(video_path):
  directory_path = f"{base_path}/images"
  create_directory(directory_path)
  var=os.system(f" ffmpeg -i {video_path} {base_path}/images/%06d.jpg")

  if var==0:
    print("We extracted frames Successfully")
    print(f"Number of Images {len(os.listdir(directory_path))}")
  else:
    print("Failed to extract frames")

import os
import cv2
from tqdm import tqdm

import os
import uuid

def generate_random_name(video_filename):
    base_name, extension = os.path.splitext(os.path.basename(video_filename))
    random_uuid = str(uuid.uuid4().hex)[:6]
    if os.path.exists(f"{base_path}/gdrive/MyDrive/upload/"):
     new_name = f"{base_path}/gdrive/MyDrive/upload/{base_name}_GFPGAN_{random_uuid}{extension}"
    else:
      dir_name=os.path.dirname(video_filename)
      if len(dir_name)!=0:
        new_name = f"{dir_name}/{base_name}_GFPGAN_{random_uuid}{extension}"
      else:
        new_name = f"{base_path}/{base_name}_GFPGAN_{random_uuid}{extension}"

    return new_name

def make_video():
    restoredFramesPath = f'{base_path}/GFPGAN/results/restored_imgs/'
    processedVideoOutputPath = f'{base_path}/output'
    create_directory(processedVideoOutputPath)
    dir_list = os.listdir(restoredFramesPath)
    dir_list.sort()

    batch = 0
    batchSize = 100
    # print(dir_list)
    for i in range(0, len(dir_list), batchSize):
        img_array = []
        start, end = i, i + batchSize
        print("processing ", start, end)
        for filename in dir_list[start:end]:
            filename = restoredFramesPath + filename
            # print(filename)
            img = cv2.imread(filename)
            if img is None:
                continue
            height, width, layers = img.shape
            size = (width, height)
            img_array.append(img)

        # Save the video as MP4
        temp_video_path=processedVideoOutputPath + f'/{str(batch).zfill(4)}.mp4'
        out = cv2.VideoWriter(temp_video_path,
                              cv2.VideoWriter_fourcc(*'mp4v'), 25, size)
        batch = batch + 1

        for i in range(len(img_array)):
            out.write(img_array[i])
        out.release()
        if os.path.exists(f"{base_path}/gdrive/MyDrive/upload/"):
          temp_drive_video_path=f"{base_path}/gdrive/MyDrive/upload" + f'/{str(batch).zfill(4)}.mp4'
          # shutil.copy(temp_video_path,temp_drive_video_path)



def update_join():
  video_folder = f'{base_path}/output'
  output_txt_file = f'{base_path}/join.txt'
  video_files = [file for file in os.listdir(video_folder) if file.endswith('.mp4')]
  video_files.sort()
  with open(output_txt_file, 'w') as file:
      for video_file in video_files:
          file.write(f"file '{os.path.join(video_folder, video_file)}'\n")

  print(f"join.txt file created at: {output_txt_file}")
  return output_txt_file
import os
import shutil
def clean_results():
  if os.path.exists(f'{base_path}/GFPGAN/results'):
    for i in os.listdir(f"{base_path}/GFPGAN/results"):
      if os.path.exists(f"{base_path}/GFPGAN/results/{i}"):
        shutil.rmtree(f"{base_path}/GFPGAN/results/{i}")
import uuid
import os
def get_final_video_name():
  generated_uuid = str(uuid.uuid4())[:8]
  video_name=f"{base_path}/{generated_uuid}.mp4"
  return video_name

def upscale_video(video_path):
  extract_frames(video_path)
  if os.getcwd()!=f"{base_path}/GFPGAN":
    os.chdir(f"{base_path}/GFPGAN")
  clean_results()
  var=os.system(f"python inference_gfpgan.py -i {base_path}/images -o results -v 1.3 -s 2 --bg_upsampler realesrgan")
  if var!=0:
    print("Image Upscale Failed")

  make_video()
  update_join()
  var1=os.system(f"ffmpeg -f concat -safe 0 -i {base_path}/join.txt -c copy {base_path}/upscale.mp4 -y")
  if var1!=0:
    print("Marged Video Failed")
  if os.path.exists(f"{base_path}/audio.wav"):
    os.remove(f"{base_path}/audio.wav")
  var2=os.system(f"ffmpeg -i {video_path} {base_path}/audio.wav -y")
  if var2!=0:
    print("ffmpeg Extract audio failed")
  if os.path.exists(f"{base_path}/final_result.mp4"):
    os.remove(f"{base_path}/final_result.mp4")
  var3=os.system(f"ffmpeg -i {base_path}/upscale.mp4 -i {base_path}/audio.wav -c:v copy -map 0:v -map 1:a -y {base_path}/final_result.mp4")
  if var3==0:
    print(f"Video Save at {base_path}/final_result.mp4")
  else:
    print("ffmpeg video and audio mix failed")

  new_filename = generate_random_name(video_path)
  if os.path.exists(f"{base_path}/final_result.mp4"):
    shutil.copy(f"{base_path}/final_result.mp4",new_filename)
  else:
    shutil.copy(f"{base_path}/upscale.mp4",new_filename)
  print(f"GFPGAN Upscale video copy save at {new_filename}")
  return new_filename
from google.colab import output
def successful():
  output.eval_js('new Audio("https://www.myinstants.com/media/sounds/anime-wow-sound-effect.mp3").play()')

def error():
  output.eval_js('new Audio("https://www.myinstants.com/media/sounds/movie_1.mp3").play()')

import re
def clean_file_name(file_path):
  file_name, file_extension = os.path.splitext(os.path.basename(file_path))
  cleaned_file_name = re.sub(r'[^a-zA-Z ]', '', file_name).replace(' ', '_').strip('_')
  cleaned_file_name = re.sub(r'_+', '_', cleaned_file_name)
  new_file_path =cleaned_file_name + file_extension
  return new_file_path
def gradio_video_upscale(input_video_path):
  clean_file=f"{base_path}/{clean_file_name(input_video_path)}"
  shutil.copy(input_video_path,clean_file)
  save_path=upscale_video(clean_file)
  return save_path,save_path,save_path

def upscale_image(img_path):
  if not os.path.exists(f"{base_path}/upscaled_image"):
    os.mkdir(f"{base_path}/upscaled_image")
  if os.getcwd()!=f"{base_path}/GFPGAN":
    os.chdir(f"{base_path}/GFPGAN")
  clean_results()
  var=os.system(f"python inference_gfpgan.py -i {img_path} -o results -v 1.3 -s 2 --bg_upsampler realesrgan")
  if var!=0:
    print("Image Upscale Failed")
    return None
  else:
    file_name = os.path.basename(img_path)
    image_path=f"{base_path}/GFPGAN/results/restored_imgs/{file_name}"
    new_path=f"{base_path}/upscaled_image/{file_name}"
    shutil.copy(image_path,new_path)
    return new_path

from zipfile import ZipFile

def zip_folder(folder_path, zip_path):
    if os.path.exists(zip_path):
      os.remove(zip_path)
    with ZipFile(zip_path, 'w') as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, folder_path)
                zipf.write(file_path, arcname=arcname)
from PIL import Image

def manage_files(multiple_images):
  if len(multiple_images)==1:
    save_path=upscale_image(multiple_images[-1])
    pil_image = Image.open(save_path)
    return save_path,save_path,pil_image
  else:
    random_uuid = str(uuid.uuid4().hex)[:6]
    os.makedirs(f"{base_path}/temp/GFPGAN_{random_uuid}")
    for image in multiple_images:
      save_path=upscale_image(image)
      file_name=os.path.basename(save_path)
      shutil.copy(save_path,f"{base_path}/temp/GFPGAN_{random_uuid}/{file_name}")
    zip_folder(f"{base_path}/temp/GFPGAN_{random_uuid}", f"{base_path}/temp/GFPGAN_{random_uuid}.zip")
    full_path = os.path.abspath(f"{base_path}/temp/GFPGAN_{random_uuid}.zip")
    pil_image = Image.open(f"{base_path}/GFPGAN/Download.png")
    return f"{base_path}/temp/GFPGAN_{random_uuid}.zip",full_path,pil_image

In [ ]:
#@title Gradio webapp for Image & Video Upscale [Optional]
import gradio as gr
image_demo_inputs=[gr.File(label="Upload Single or Multiple Images",file_count="multiple",file_types=['image'],type='filepath')]
image_demo_outputs=[gr.File(label="Download Image"),gr.Textbox(label="Download From Colab Full Path is"),gr.Image(type="pil",label="Display Image")]
image_demo = gr.Interface(fn=manage_files, inputs=image_demo_inputs,outputs=image_demo_outputs , title="GFPGAN 1.4 Image Upscale")
video_demo_inputs=[gr.File(label="Upload a Video",file_types=['.mp4'],type='filepath')]
video_demo_outputs=[gr.File(label="Download Video"),gr.Textbox(label="Download From Colab Full Path is"),gr.Video(label="Display Video")]
video_demo = gr.Interface(fn=gradio_video_upscale, inputs=video_demo_inputs,outputs=video_demo_outputs , title="GFPGAN 1.4 Video Upscale")
demo = gr.TabbedInterface([image_demo,video_demo], ["GFPGAN 1.4 Image Upscale", "GFPGAN 1.4 Video Upscale"],title="GFPGAN 1.4 Image & Video Upscale")
if __name__ == "__main__":
    demo.launch(allowed_paths=[f"{base_path}/upscaled_image",base_path],debug=False)

# Without Gradio webapp

In [ ]:
# @title ##**Mount drive for faster video upload, download and backup [For Colab]** { display-mode: "form" }
base_path="/content"
from IPython.display import clear_output
from google.colab import drive
import os
import shutil
Mount_Google_Drive = True #@param {type:"boolean"}
if Mount_Google_Drive:
  drive.mount(f'{base_path}/gdrive')
  clear_output()
  if os.path.exists(f"{base_path}/gdrive/MyDrive/"):
    if not os.path.exists(f"{base_path}/gdrive/MyDrive/upload/"):
      os.mkdir(f"{base_path}/gdrive/MyDrive/upload/")
    print("Open Google Drive and upload video in the folder called 'upload' or Use as auto backup")

Open Google Drive and upload video in the folder called 'upload' or Use as auto backup


In [ ]:
# @title ##**Choose option [For Colab]** { display-mode: "form" }
import os
import re
import shutil
base_path="/content"
from IPython.display import clear_output

def get_clean_name(file_path):
  # Extract file name and extension
  file_name, file_extension = os.path.splitext(os.path.basename(file_path))
  # Clean and optimize the file name
  cleaned_file_name = re.sub(r'[^a-zA-Z ]', '', file_name).replace(' ', '_').strip('_')
  # Replace consecutive underscores with a single underscore
  cleaned_file_name = re.sub(r'_+', '_', cleaned_file_name)
  # Create the new file path
  new_file_path = os.path.join(os.path.dirname(file_path), cleaned_file_name + file_extension)
  return new_file_path

choose= "drive_uploaded"#@param ['upload_now','drive_uploaded']
if not os.path.exists(f"{base_path}/upload/"):
  os.mkdir(f"{base_path}/upload/")
clear_output()
f_names=[]
from google.colab import files
if choose== "upload_now":
  os.chdir(f"{base_path}/upload/")
  uploaded = files.upload()
  for fn in uploaded.keys():
        file_path=f"{base_path}/upload/{fn}"
        new_file_path=get_clean_name(file_path)
        if not os.path.exists(new_file_path):
          shutil.copy(file_path,new_file_path)
        f_names.append(new_file_path)
  clear_output()
  print(f_names[-1])
if choose== "drive_uploaded":
  video_id=1
  table={}
  ids=[]
  videos=[]
  id_monitor={}
  video_folder=f"{base_path}/gdrive/MyDrive/upload"
  if not os.path.exists(video_folder):
    print(f"Run above cell to Mount google drive")
  else:
    for i in os.listdir(video_folder):
      ids.append(video_id)
      videos.append(video_folder+"/"+i)
      id_monitor[video_id]=video_folder+"/"+i
      video_id=video_id+1
    table["video_name"]=videos
    table["video_id"]=ids
    import pandas as pd
    df = pd.DataFrame(table)
    try:
      df.set_index('video_id', inplace=True)
    except:
      pass
    print(df)
    print("\n")

                                                 video_name
video_id                                                   
1                     /content/gdrive/MyDrive/upload/me.mp4
2         /content/gdrive/MyDrive/upload/.ipynb_checkpoints
3              /content/gdrive/MyDrive/upload/Elon Musk.jpg
4         /content/gdrive/MyDrive/upload/81945-577442929...
5                    /content/gdrive/MyDrive/upload/lip.mp4




In [ ]:
# @title ##**If you selected 'drive_uploaded' enter video id else just run  [For Colab]** { display-mode: "form" }

if choose== "upload_now":
  pass
if choose== "drive_uploaded":
  video_id =   5#@param {type: "number"}
  f_names.append(id_monitor[int(video_id)])
f_names[-1]

'/content/gdrive/MyDrive/upload/lip.mp4'

In [ ]:
#@title Enter a Video path
def clean_file_name(file_path):
  file_name, file_extension = os.path.splitext(os.path.basename(file_path))
  cleaned_file_name = re.sub(r'[^a-zA-Z ]', '', file_name).replace(' ', '_').strip('_')
  cleaned_file_name = re.sub(r'_+', '_', cleaned_file_name)
  new_file_path =cleaned_file_name + file_extension
  return new_file_path

input_video_path = '/content/gdrive/MyDrive/upload/lip.mp4'  # @param {type: "string"}
clean_file=f"{base_path}/{clean_file_name(input_video_path)}"
shutil.copy(input_video_path,clean_file)
save_path=upscale_video(clean_file)
successful()
from google.colab import files
files.download(save_path)

We extracted frames Successfully
Number of Images 1024
processing  0 600


In [ ]:
import shutil
import os

def zip_folder(folder_path, output_path):
    # Create a zip file from the specified folder
    shutil.make_archive(output_path, 'zip', folder_path)
    # Add .zip extension to the output path
    zip_path = output_path + '.zip'
    return zip_path

# Example usage
folder_path = '/content/GFPGAN/results/restored_imgs'
output_path = '/content/gdrive/MyDrive/image.zip'

zip_file_path = zip_folder(folder_path, output_path)
print(f'Folder zipped to: {zip_file_path}')
